In [2]:
from sqlalchemy import create_engine
import pandas as pd
import re 

def connect_db(dico):

    str_engine = "mysql+" + dico["connector"] + "://" + dico["user"] + ":" + dico["pwd"] + "@" + dico["host"] + ":" + dico["port"] + "/" + dico["bdd"]
    connect = create_engine(str_engine)
    return connect

d = {
    "connector": "pymysql",
    "user" : "simplon",
    "pwd" : "fidelio",
    "host" : "localhost",
    "port": "3306",
    "bdd" : "survey"
}

engine = connect_db(d)
# https://docs.sqlalchemy.org/en/14/core/engines.html

In [3]:
survey_data = pd.read_excel("/home/simplon/Simplon_dev_IA/jeux_de_données/Data_Professional_Salary_Survey_Responses_1.xlsx")
survey_data
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html

,Survey Year,Timestamp,SalaryUSD,Country,PostalCode,PrimaryDatabase,YearsWithThisDatabase,OtherDatabases,EmploymentStatus,JobTitle,...,NewestVersionInProduction,OldestVersionInProduction,PopulationOfLargestCityWithin20Miles,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties,KindsOfTasksPerformed,Counter
0,2021,2020-12-10 08:22:42.696,65000.0,Sweden,NaN,Microsoft SQL Server,4,Microsoft SQL Server,Full time employee,"Developer: Business Intelligence (SSRS, PowerB...",...,SQL Server 2016,SQL Server 2016,"<= 20,000 (town)",Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked,Not Asked,1
1,2021,2020-12-10 08:23:22.242,145000.0,United States,76063,Microsoft SQL Server,15,Azure SQL DB (any flavor),Full time employee,DBA (Production Focus - build & troubleshoot s...,...,SQL Server 2019,SQL Server 2014,1M+ (metropolis),Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked,Not Asked,1
2,2021,2020-12-10 08:23:37.909,105000.0,United States,43240,Microsoft SQL Server,12,"PostgreSQL, Azure SQL DB (any flavor)",Full time employee,DBA (General - splits time evenly between writ...,...,SQL Server 2017,SQL Server 2008R2,300K-1M (large city),Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked,Not Asked,1
3,2021,2020-12-10 08:23:48.152,46482.0,United Kingdom,NaN,Microsoft SQL Server,10,Azure SQL DB (any flavor),Full time employee,DBA (Production Focus - build & troubleshoot s...,...,SQL Server 2019,SQL Server 2012,300K-1M (large city),"Education (K-12, college, university)",Not Asked,"Stay with the same employer, same role",Male,Not Asked,Not Asked,1
4,2021,2020-12-10 08:24:03.600,98800.0,United States,468,Microsoft SQL Server,5,DB2,Full time employee,"Developer: Business Intelligence (SSRS, PowerB...",...,SQL Server 2019,SQL Server 2012,100K-299K (city),Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked,Not Asked,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10336,2017,2017-01-14 15:50:23.936,36549.0,United Kingdom,Not Asked,Microsoft SQL Server,3,NaN,Full time employee,DBA,...,Not Asked,Not Asked,Not Asked,Private business,No,Not Asked,Not Asked,Not Asked,Not Asked,1
10337,2017,2017-01-15 01:01:21.212,65000.0,Saudi Arabia,Not Asked,Microsoft SQL Server,7,"Oracle, MySQL/MariaDB, Microsoft Access, SQLite",Full time employee,DBA,...,Not Asked,Not Asked,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
10338,2017,2017-01-15 03:25:47.422,85000.0,Netherlands,Not Asked,Microsoft SQL Server,17,Oracle,Full time employee,Other,...,Not Asked,Not Asked,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
10339,2017,2017-01-15 03:37:35.305,90000.0,United States,Not Asked,Microsoft SQL Server,8,MySQL/MariaDB,Full time employee,DBA,...,Not Asked,Not Asked,Not Asked,Private business,"Yes, actively looking for something else",Not Asked,Not Asked,Not Asked,Not Asked,1


# Création DataFrames pour insertion dans la base de données

# DataFrame sondage_item

In [4]:
sondage_item_dic = {
    
    "timestamp": "Timestamp",
    "salary_usd" : "SalaryUSD",
    "postal_code" : "PostalCode",
    "years_with_db" : "YearsWithThisDatabase",
    "manage_staff": "ManageStaff",
    "years_with_job" : "YearsWithThisTypeOfJob",
    "other_people": "OtherPeopleOnYourTeam",
    "company_employee" : "CompanyEmployeesOverall",
    "database_servers" : "DatabaseServers",
    "education_computer" : "EducationIsComputerRelated",
    "hours_worked": "HoursWorkedPerWeek",
    "telecommute" : "TelecommuteDaysPerWeek",
    "newest_version": "NewestVersionInProduction",
    "oldest_version" : "OldestVersionInProduction",
    "gender" : "Gender"

}

sondage_item_df = pd.DataFrame(columns= [i for i in sondage_item_dic.keys()])
sondage_item_df

# Settage des index

#sondage_item_df = sondage_item_df.rename_axis(index="sgi_id")
sondage_item_df["sgi_id"] = range(1, sondage_item_df.shape[0] + 1)
sondage_item_df

for i in sondage_item_dic.keys():
   sondage_item_df[str(i)] = survey_data[str(sondage_item_dic[i])]

# DataFrame carrer_plan

In [5]:
carrer_plan_columns = pd.read_sql_query("select * from career_plan;", engine).columns
carrer_plan_columns

Index(['cap_id', 'cap_plan'], dtype='object')

In [6]:
carrer_df = survey_data["CareerPlansThisYear"]
carrer_df.drop_duplicates(inplace=True)
carrer_df = carrer_df.reset_index(drop=True)
carrer_df.index += 1

carrer_df = pd.DataFrame({"cap_plan" : carrer_df})
#carrer_df = carrer_df.rename_axis(index="cap_id")
carrer_df["cap_id"] = carrer_df.index
carrer_df

,cap_plan,cap_id
1,"Stay with the same employer, same role",1
2,Prefer not to say,2
3,Change both employers and roles,3
4,"Stay with the same role, but change employers",4
5,"Stay with the same employer, but change roles",5
6,Not Asked,6


In [7]:
id_carrer = survey_data["CareerPlansThisYear"].apply(lambda x: carrer_df[carrer_df["cap_plan"] == x].index[0])

sondage_item_df["cap_id"] = id_carrer.astype(pd.Int64Dtype())
sondage_item_df

,timestamp,salary_usd,postal_code,years_with_db,manage_staff,years_with_job,other_people,company_employee,database_servers,education_computer,hours_worked,telecommute,newest_version,oldest_version,gender,sgi_id,cap_id
0,2020-12-10 08:22:42.696,65000.0,NaN,4,Yes,4,None,2,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2016,SQL Server 2016,Male,NaN,1
1,2020-12-10 08:23:22.242,145000.0,76063,15,No,25,None,74,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2014,Male,NaN,1
2,2020-12-10 08:23:37.909,105000.0,43240,12,Yes,6,1,500,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2017,SQL Server 2008R2,Male,NaN,1
3,2020-12-10 08:23:48.152,46482.0,NaN,10,No,5,None,35,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2012,Male,NaN,1
4,2020-12-10 08:24:03.600,98800.0,468,5,No,5,None,50,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2012,Male,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10336,2017-01-14 15:50:23.936,36549.0,Not Asked,3,No,1,1,Not Asked,42,NaN,38,"None, or less than 1 day per week",Not Asked,Not Asked,Not Asked,NaN,6
10337,2017-01-15 01:01:21.212,65000.0,Not Asked,7,Yes,3,5,Not Asked,40,Yes,50,1,Not Asked,Not Asked,Not Asked,NaN,6
10338,2017-01-15 03:25:47.422,85000.0,Not Asked,17,No,17,1,Not Asked,200,No,40,1,Not Asked,Not Asked,Not Asked,NaN,6
10339,2017-01-15 03:37:35.305,90000.0,Not Asked,8,No,3,1,Not Asked,80,Yes,45,5 or more,Not Asked,Not Asked,Not Asked,NaN,6


# DataFrame certification

In [8]:
certification_columns = pd.read_sql_query("select * from certification;", engine).columns
certification_columns

Index(['cert_id', 'cert_name'], dtype='object')

In [9]:
certification_df = survey_data["Certifications"]
certification_df.drop_duplicates(inplace=True)
certification_df = certification_df.reset_index(drop=True)
certification_df.index += 1

certification_df = pd.DataFrame({"cert_name" : certification_df})
#certification_df = certification_df.rename_axis(index="cert_id")
certification_df["cert_id"] = certification_df.index
certification_df

,cert_name,cert_id
1,Not Asked,1
2,"No, I never have",2
3,"Yes, and they're currently valid",3
4,"Yes, but they expired",4


In [10]:
id_certification = survey_data["Certifications"].apply(lambda x: certification_df[certification_df["cert_name"] == x].index[0])

sondage_item_df["cert_id"] = id_certification.astype(pd.Int64Dtype())
sondage_item_df

,timestamp,salary_usd,postal_code,years_with_db,manage_staff,years_with_job,other_people,company_employee,database_servers,education_computer,hours_worked,telecommute,newest_version,oldest_version,gender,sgi_id,cap_id,cert_id
0,2020-12-10 08:22:42.696,65000.0,NaN,4,Yes,4,None,2,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2016,SQL Server 2016,Male,NaN,1,1
1,2020-12-10 08:23:22.242,145000.0,76063,15,No,25,None,74,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2014,Male,NaN,1,1
2,2020-12-10 08:23:37.909,105000.0,43240,12,Yes,6,1,500,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2017,SQL Server 2008R2,Male,NaN,1,1
3,2020-12-10 08:23:48.152,46482.0,NaN,10,No,5,None,35,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2012,Male,NaN,1,1
4,2020-12-10 08:24:03.600,98800.0,468,5,No,5,None,50,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2012,Male,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10336,2017-01-14 15:50:23.936,36549.0,Not Asked,3,No,1,1,Not Asked,42,NaN,38,"None, or less than 1 day per week",Not Asked,Not Asked,Not Asked,NaN,6,2
10337,2017-01-15 01:01:21.212,65000.0,Not Asked,7,Yes,3,5,Not Asked,40,Yes,50,1,Not Asked,Not Asked,Not Asked,NaN,6,2
10338,2017-01-15 03:25:47.422,85000.0,Not Asked,17,No,17,1,Not Asked,200,No,40,1,Not Asked,Not Asked,Not Asked,NaN,6,2
10339,2017-01-15 03:37:35.305,90000.0,Not Asked,8,No,3,1,Not Asked,80,Yes,45,5 or more,Not Asked,Not Asked,Not Asked,NaN,6,3


# DataFrame country

In [11]:
country_columns = pd.read_sql_query("select * from country;", engine).columns
country_columns

Index(['ctr_id', 'ctr_name'], dtype='object')

In [12]:
country_df = survey_data["Country"]
country_df.drop_duplicates(inplace=True)
country_df = country_df.reset_index(drop=True)
country_df.index += 1

country_df = pd.DataFrame({"ctr_name" : country_df})
#country_df = country_df.rename_axis(index="ctr_id")
country_df["ctr_id"] = country_df.index
country_df

,ctr_name,ctr_id
1,Sweden,1
2,United States,2
3,United Kingdom,3
4,Pakistan,4
5,Paraguay,5
...,...,...
93,Albania,93
94,Bolivia,94
95,Vietnam,95
96,Bermuda,96


In [13]:
id_country = survey_data["Country"].apply(lambda x: country_df[country_df["ctr_name"] == x].index[0])

sondage_item_df["ctr_id"] = id_country.astype(pd.Int64Dtype())
sondage_item_df

,timestamp,salary_usd,postal_code,years_with_db,manage_staff,years_with_job,other_people,company_employee,database_servers,education_computer,hours_worked,telecommute,newest_version,oldest_version,gender,sgi_id,cap_id,cert_id,ctr_id
0,2020-12-10 08:22:42.696,65000.0,NaN,4,Yes,4,None,2,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2016,SQL Server 2016,Male,NaN,1,1,1
1,2020-12-10 08:23:22.242,145000.0,76063,15,No,25,None,74,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2014,Male,NaN,1,1,2
2,2020-12-10 08:23:37.909,105000.0,43240,12,Yes,6,1,500,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2017,SQL Server 2008R2,Male,NaN,1,1,2
3,2020-12-10 08:23:48.152,46482.0,NaN,10,No,5,None,35,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2012,Male,NaN,1,1,3
4,2020-12-10 08:24:03.600,98800.0,468,5,No,5,None,50,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2012,Male,NaN,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10336,2017-01-14 15:50:23.936,36549.0,Not Asked,3,No,1,1,Not Asked,42,NaN,38,"None, or less than 1 day per week",Not Asked,Not Asked,Not Asked,NaN,6,2,3
10337,2017-01-15 01:01:21.212,65000.0,Not Asked,7,Yes,3,5,Not Asked,40,Yes,50,1,Not Asked,Not Asked,Not Asked,NaN,6,2,73
10338,2017-01-15 03:25:47.422,85000.0,Not Asked,17,No,17,1,Not Asked,200,No,40,1,Not Asked,Not Asked,Not Asked,NaN,6,2,28
10339,2017-01-15 03:37:35.305,90000.0,Not Asked,8,No,3,1,Not Asked,80,Yes,45,5 or more,Not Asked,Not Asked,Not Asked,NaN,6,3,2


# DataFrame education

In [14]:
education_columns = pd.read_sql_query("select * from education;", engine).columns
education_columns

Index(['edu_id', 'edu_title'], dtype='object')

In [15]:
education_df = survey_data["Education"]
education_df.drop_duplicates(inplace=True)
education_df = education_df.reset_index(drop=True)
education_df.index += 1

education_df = pd.DataFrame({"edu_title" : education_df})
#education_df = education_df.rename_axis(index="edu_id")
education_df["edu_id"] = education_df.index
education_df

,edu_title,edu_id
1,Not Asked,1
2,Bachelors (4 years),2
3,None (no degree completed),3
4,Masters,4
5,Associates (2 years),5
6,Doctorate/PhD,6


In [16]:
id_education = survey_data["Education"].apply(lambda x: education_df[education_df["edu_title"] == x].index[0])

sondage_item_df["edu_id"] = id_education.astype(pd.Int64Dtype())
sondage_item_df

,timestamp,salary_usd,postal_code,years_with_db,manage_staff,years_with_job,other_people,company_employee,database_servers,education_computer,hours_worked,telecommute,newest_version,oldest_version,gender,sgi_id,cap_id,cert_id,ctr_id,edu_id
0,2020-12-10 08:22:42.696,65000.0,NaN,4,Yes,4,None,2,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2016,SQL Server 2016,Male,NaN,1,1,1,1
1,2020-12-10 08:23:22.242,145000.0,76063,15,No,25,None,74,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2014,Male,NaN,1,1,2,1
2,2020-12-10 08:23:37.909,105000.0,43240,12,Yes,6,1,500,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2017,SQL Server 2008R2,Male,NaN,1,1,2,1
3,2020-12-10 08:23:48.152,46482.0,NaN,10,No,5,None,35,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2012,Male,NaN,1,1,3,1
4,2020-12-10 08:24:03.600,98800.0,468,5,No,5,None,50,Not Asked,Not Asked,Not Asked,Not Asked,SQL Server 2019,SQL Server 2012,Male,NaN,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10336,2017-01-14 15:50:23.936,36549.0,Not Asked,3,No,1,1,Not Asked,42,NaN,38,"None, or less than 1 day per week",Not Asked,Not Asked,Not Asked,NaN,6,2,3,3
10337,2017-01-15 01:01:21.212,65000.0,Not Asked,7,Yes,3,5,Not Asked,40,Yes,50,1,Not Asked,Not Asked,Not Asked,NaN,6,2,73,2
10338,2017-01-15 03:25:47.422,85000.0,Not Asked,17,No,17,1,Not Asked,200,No,40,1,Not Asked,Not Asked,Not Asked,NaN,6,2,28,2
10339,2017-01-15 03:37:35.305,90000.0,Not Asked,8,No,3,1,Not Asked,80,Yes,45,5 or more,Not Asked,Not Asked,Not Asked,NaN,6,3,2,4


# DataFrame employment_sector

In [17]:
employment_sector_columns = pd.read_sql_query("select * from employment_sector;", engine).columns
employment_sector_columns

Index(['sec_id', 'sec_name'], dtype='object')

In [18]:
employment_sector_df = survey_data["EmploymentSector"]
employment_sector_df.drop_duplicates(inplace=True)
employment_sector_df = employment_sector_df.reset_index(drop=True)
employment_sector_df.index += 1

employment_sector_df = pd.DataFrame({"sec_name" : employment_sector_df})
#employment_sector_df = employment_sector_df.rename_axis(index="sec_id")
employment_sector_df["sec_id"] = employment_sector_df.index
employment_sector_df

,sec_name,sec_id
1,Private business,1
2,"Education (K-12, college, university)",2
3,State/province government,3
4,Local government,4
5,Non-profit,5
6,Federal government,6
7,Student,7


In [19]:
id_employment_sector = survey_data["EmploymentSector"].apply(lambda x: employment_sector_df[employment_sector_df["sec_name"] == x].index[0])

sondage_item_df["sec_id"] = id_employment_sector.astype(pd.Int64Dtype())
sondage_item_df

,timestamp,salary_usd,postal_code,years_with_db,manage_staff,years_with_job,other_people,company_employee,database_servers,education_computer,...,telecommute,newest_version,oldest_version,gender,sgi_id,cap_id,cert_id,ctr_id,edu_id,sec_id
0,2020-12-10 08:22:42.696,65000.0,NaN,4,Yes,4,None,2,Not Asked,Not Asked,...,Not Asked,SQL Server 2016,SQL Server 2016,Male,NaN,1,1,1,1,1
1,2020-12-10 08:23:22.242,145000.0,76063,15,No,25,None,74,Not Asked,Not Asked,...,Not Asked,SQL Server 2019,SQL Server 2014,Male,NaN,1,1,2,1,1
2,2020-12-10 08:23:37.909,105000.0,43240,12,Yes,6,1,500,Not Asked,Not Asked,...,Not Asked,SQL Server 2017,SQL Server 2008R2,Male,NaN,1,1,2,1,1
3,2020-12-10 08:23:48.152,46482.0,NaN,10,No,5,None,35,Not Asked,Not Asked,...,Not Asked,SQL Server 2019,SQL Server 2012,Male,NaN,1,1,3,1,2
4,2020-12-10 08:24:03.600,98800.0,468,5,No,5,None,50,Not Asked,Not Asked,...,Not Asked,SQL Server 2019,SQL Server 2012,Male,NaN,1,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10336,2017-01-14 15:50:23.936,36549.0,Not Asked,3,No,1,1,Not Asked,42,NaN,...,"None, or less than 1 day per week",Not Asked,Not Asked,Not Asked,NaN,6,2,3,3,1
10337,2017-01-15 01:01:21.212,65000.0,Not Asked,7,Yes,3,5,Not Asked,40,Yes,...,1,Not Asked,Not Asked,Not Asked,NaN,6,2,73,2,1
10338,2017-01-15 03:25:47.422,85000.0,Not Asked,17,No,17,1,Not Asked,200,No,...,1,Not Asked,Not Asked,Not Asked,NaN,6,2,28,2,1
10339,2017-01-15 03:37:35.305,90000.0,Not Asked,8,No,3,1,Not Asked,80,Yes,...,5 or more,Not Asked,Not Asked,Not Asked,NaN,6,3,2,4,1


# DataFrame employment_status

In [20]:
employment_status_columns = pd.read_sql_query("select * from employment_status;", engine).columns
employment_status_columns

Index(['emp_id', 'emp_status'], dtype='object')

In [21]:
employment_status_df = survey_data["EmploymentStatus"]
employment_status_df.drop_duplicates(inplace=True)
employment_status_df = employment_status_df.reset_index(drop=True)
employment_status_df.index += 1

employment_status_df = pd.DataFrame({"emp_status" : employment_status_df})
#employment_status_df = employment_status_df.rename_axis(index="emp_id")
employment_status_df["emp_id"] = employment_status_df.index
employment_status_df

,emp_status,emp_id
1,Full time employee,1
2,Full time employee of a consulting/contracting...,2
3,"Independent consultant, contractor, freelancer...",3
4,Part time,4
5,Independent or freelancer or company owner,5


In [22]:
id_employment_status = survey_data["EmploymentStatus"].apply(lambda x: employment_status_df[employment_status_df["emp_status"] == x].index[0])

id_employment_status["emp_id"] = id_employment_status.astype(pd.Int64Dtype())
id_employment_status

0                                                         1
1                                                         1
2                                                         1
3                                                         1
4                                                         1
                                ...                        
10337                                                     1
10338                                                     1
10339                                                     1
10340                                                     1
emp_id    0        1
1        1
2        1
3        1
4 ...
Name: EmploymentStatus, Length: 10342, dtype: object

# DataFrame how_many_companies

In [23]:
column_how_many_companies = pd.read_sql_query("select * from how_many_companies;", engine).columns
column_how_many_companies

Index(['mcp_id', 'mcp_many_companies'], dtype='object')

In [24]:
hmc_df = survey_data["HowManyCompanies"]
hmc_df.drop_duplicates(inplace=True)
hmc_df = hmc_df.reset_index(drop=True)
hmc_df.index += 1

hmc_df = pd.DataFrame({"mcp_many_companies" : hmc_df})
#hmc_df = hmc_df.rename_axis(index="mcp_id")
hmc_df["mcp_id"] = hmc_df.index
hmc_df

,mcp_many_companies,mcp_id
1,1,1
2,5,2
3,4,3
4,2,4
5,3,5
6,6 or more,6
7,Not Asked,7


In [25]:
id_hmc = survey_data["HowManyCompanies"].apply(lambda x: hmc_df[hmc_df["mcp_many_companies"] == x].index[0])

sondage_item_df["mcp_id"] = id_hmc.astype(pd.Int64Dtype())

# DataFrame largest_city

In [26]:
largest_city_companies = pd.read_sql_query("select * from largest_city;", engine).columns
largest_city_companies

Index(['pop_id', 'pop_name'], dtype='object')

In [27]:
largest_city_df = survey_data["PopulationOfLargestCityWithin20Miles"]
largest_city_df.drop_duplicates(inplace=True)
largest_city_df = largest_city_df.reset_index(drop=True)
largest_city_df.index += 1

largest_city_df = pd.DataFrame({"pop_name" : largest_city_df})
#largest_city_df = largest_city_df.rename_axis(index="pop_id")
largest_city_df["pop_id"] = largest_city_df.index
largest_city_df

,pop_name,pop_id
1,"<= 20,000 (town)",1
2,1M+ (metropolis),2
3,300K-1M (large city),3
4,100K-299K (city),4
5,20K-99K (large town),5
6,Not Asked,6


In [28]:
id_lc = survey_data["PopulationOfLargestCityWithin20Miles"].apply(lambda x: largest_city_df[largest_city_df["pop_name"] == x].index[0])

sondage_item_df["pop_id"] = id_lc.astype(pd.Int64Dtype())
sondage_item_df

,timestamp,salary_usd,postal_code,years_with_db,manage_staff,years_with_job,other_people,company_employee,database_servers,education_computer,...,oldest_version,gender,sgi_id,cap_id,cert_id,ctr_id,edu_id,sec_id,mcp_id,pop_id
0,2020-12-10 08:22:42.696,65000.0,NaN,4,Yes,4,None,2,Not Asked,Not Asked,...,SQL Server 2016,Male,NaN,1,1,1,1,1,1,1
1,2020-12-10 08:23:22.242,145000.0,76063,15,No,25,None,74,Not Asked,Not Asked,...,SQL Server 2014,Male,NaN,1,1,2,1,1,2,2
2,2020-12-10 08:23:37.909,105000.0,43240,12,Yes,6,1,500,Not Asked,Not Asked,...,SQL Server 2008R2,Male,NaN,1,1,2,1,1,3,3
3,2020-12-10 08:23:48.152,46482.0,NaN,10,No,5,None,35,Not Asked,Not Asked,...,SQL Server 2012,Male,NaN,1,1,3,1,2,4,3
4,2020-12-10 08:24:03.600,98800.0,468,5,No,5,None,50,Not Asked,Not Asked,...,SQL Server 2012,Male,NaN,1,1,2,1,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10336,2017-01-14 15:50:23.936,36549.0,Not Asked,3,No,1,1,Not Asked,42,NaN,...,Not Asked,Not Asked,NaN,6,2,3,3,1,7,6
10337,2017-01-15 01:01:21.212,65000.0,Not Asked,7,Yes,3,5,Not Asked,40,Yes,...,Not Asked,Not Asked,NaN,6,2,73,2,1,7,6
10338,2017-01-15 03:25:47.422,85000.0,Not Asked,17,No,17,1,Not Asked,200,No,...,Not Asked,Not Asked,NaN,6,2,28,2,1,7,6
10339,2017-01-15 03:37:35.305,90000.0,Not Asked,8,No,3,1,Not Asked,80,Yes,...,Not Asked,Not Asked,NaN,6,3,2,4,1,7,6


# DataFrame looking_job

In [29]:
looking_job_companies = pd.read_sql_query("select * from looking_job;", engine).columns
looking_job_companies

Index(['look_id', 'look_job'], dtype='object')

In [30]:
looking_job_df = survey_data["LookingForAnotherJob"]
looking_job_df.drop_duplicates(inplace=True)
looking_job_df = looking_job_df.reset_index(drop=True)
looking_job_df.index += 1

looking_job_df = pd.DataFrame({"look_job" : looking_job_df})
#looking_job_df = looking_job_df.rename_axis(index="look_id")
looking_job_df["look_id"] = looking_job_df.index
looking_job_df

,look_job,look_id
1,Not Asked,1
2,No,2
3,"Yes, but only passively (just curious)",3
4,"Yes, actively looking for something else",4


In [31]:
id_looking_job = survey_data["LookingForAnotherJob"].apply(lambda x: looking_job_df[looking_job_df["look_job"] == x].index[0])

sondage_item_df["look_id"] = id_looking_job.astype(pd.Int64Dtype())
sondage_item_df

,timestamp,salary_usd,postal_code,years_with_db,manage_staff,years_with_job,other_people,company_employee,database_servers,education_computer,...,gender,sgi_id,cap_id,cert_id,ctr_id,edu_id,sec_id,mcp_id,pop_id,look_id
0,2020-12-10 08:22:42.696,65000.0,NaN,4,Yes,4,None,2,Not Asked,Not Asked,...,Male,NaN,1,1,1,1,1,1,1,1
1,2020-12-10 08:23:22.242,145000.0,76063,15,No,25,None,74,Not Asked,Not Asked,...,Male,NaN,1,1,2,1,1,2,2,1
2,2020-12-10 08:23:37.909,105000.0,43240,12,Yes,6,1,500,Not Asked,Not Asked,...,Male,NaN,1,1,2,1,1,3,3,1
3,2020-12-10 08:23:48.152,46482.0,NaN,10,No,5,None,35,Not Asked,Not Asked,...,Male,NaN,1,1,3,1,2,4,3,1
4,2020-12-10 08:24:03.600,98800.0,468,5,No,5,None,50,Not Asked,Not Asked,...,Male,NaN,1,1,2,1,1,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10336,2017-01-14 15:50:23.936,36549.0,Not Asked,3,No,1,1,Not Asked,42,NaN,...,Not Asked,NaN,6,2,3,3,1,7,6,2
10337,2017-01-15 01:01:21.212,65000.0,Not Asked,7,Yes,3,5,Not Asked,40,Yes,...,Not Asked,NaN,6,2,73,2,1,7,6,3
10338,2017-01-15 03:25:47.422,85000.0,Not Asked,17,No,17,1,Not Asked,200,No,...,Not Asked,NaN,6,2,28,2,1,7,6,3
10339,2017-01-15 03:37:35.305,90000.0,Not Asked,8,No,3,1,Not Asked,80,Yes,...,Not Asked,NaN,6,3,2,4,1,7,6,4


# DataFrame sondage

In [32]:
sondage_columns = pd.read_sql_query("select * from sondage;", engine).columns
sondage_columns

Index(['sdg_id', 'sdg_year'], dtype='object')

In [33]:
sondage_df = survey_data["Survey Year"]
sondage_df.drop_duplicates(inplace=True)
sondage_df = sondage_df.reset_index(drop=True)
sondage_df.index += 1

sondage_df = pd.DataFrame({"sdg_year" : sondage_df})
#sondage_df = sondage_df.rename_axis(index="sdg_id")
sondage_df["sdg_id"] = range(1, sondage_df.shape[0] + 1)
sondage_df

,sdg_year,sdg_id
1,2021,1
2,2020,2
3,2019,3
4,2018,4
5,2017,5


In [34]:
id_sondage = survey_data["Survey Year"].apply(lambda x: sondage_df[sondage_df["sdg_year"] == x].index[0])

sondage_item_df["sdg_id"] = id_sondage.astype(pd.Int64Dtype())
sondage_item_df

,timestamp,salary_usd,postal_code,years_with_db,manage_staff,years_with_job,other_people,company_employee,database_servers,education_computer,...,sgi_id,cap_id,cert_id,ctr_id,edu_id,sec_id,mcp_id,pop_id,look_id,sdg_id
0,2020-12-10 08:22:42.696,65000.0,NaN,4,Yes,4,None,2,Not Asked,Not Asked,...,NaN,1,1,1,1,1,1,1,1,1
1,2020-12-10 08:23:22.242,145000.0,76063,15,No,25,None,74,Not Asked,Not Asked,...,NaN,1,1,2,1,1,2,2,1,1
2,2020-12-10 08:23:37.909,105000.0,43240,12,Yes,6,1,500,Not Asked,Not Asked,...,NaN,1,1,2,1,1,3,3,1,1
3,2020-12-10 08:23:48.152,46482.0,NaN,10,No,5,None,35,Not Asked,Not Asked,...,NaN,1,1,3,1,2,4,3,1,1
4,2020-12-10 08:24:03.600,98800.0,468,5,No,5,None,50,Not Asked,Not Asked,...,NaN,1,1,2,1,1,1,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10336,2017-01-14 15:50:23.936,36549.0,Not Asked,3,No,1,1,Not Asked,42,NaN,...,NaN,6,2,3,3,1,7,6,2,5
10337,2017-01-15 01:01:21.212,65000.0,Not Asked,7,Yes,3,5,Not Asked,40,Yes,...,NaN,6,2,73,2,1,7,6,3,5
10338,2017-01-15 03:25:47.422,85000.0,Not Asked,17,No,17,1,Not Asked,200,No,...,NaN,6,2,28,2,1,7,6,3,5
10339,2017-01-15 03:37:35.305,90000.0,Not Asked,8,No,3,1,Not Asked,80,Yes,...,NaN,6,3,2,4,1,7,6,4,5


---

# DataFrame job

In [35]:
job_columns = pd.read_sql_query("select * from job;", engine).columns
job_columns

Index(['job_id', 'job_name'], dtype='object')

In [36]:
job_df = survey_data["JobTitle"]
job_df.drop_duplicates(inplace=True)
job_df = job_df.reset_index(drop=True)
job_df.index += 1

job_df = pd.DataFrame({"job_name" : job_df})
#job_df = job_df.rename_axis(index="job_id")
job_df["job_id"] = job_df.index
job_df

,job_name,job_id
1,"Developer: Business Intelligence (SSRS, PowerB...",1
2,DBA (Production Focus - build & troubleshoot s...,2
3,DBA (General - splits time evenly between writ...,3
4,Manager,4
5,"Developer: App code (C#, JS, etc)",5
6,Developer: T-SQL,6
7,Architect,7
8,"DBA (Development Focus - tunes queries, indexe...",8
9,Engineer,9
10,Analyst,10


In [37]:
id_jobtitle = survey_data["JobTitle"].apply(lambda x: job_df[job_df["job_name"] == x].index[0])

sondage_item_df["job_id"] = id_jobtitle.astype(pd.Int64Dtype())
sondage_item_df

,timestamp,salary_usd,postal_code,years_with_db,manage_staff,years_with_job,other_people,company_employee,database_servers,education_computer,...,cap_id,cert_id,ctr_id,edu_id,sec_id,mcp_id,pop_id,look_id,sdg_id,job_id
0,2020-12-10 08:22:42.696,65000.0,NaN,4,Yes,4,None,2,Not Asked,Not Asked,...,1,1,1,1,1,1,1,1,1,1
1,2020-12-10 08:23:22.242,145000.0,76063,15,No,25,None,74,Not Asked,Not Asked,...,1,1,2,1,1,2,2,1,1,2
2,2020-12-10 08:23:37.909,105000.0,43240,12,Yes,6,1,500,Not Asked,Not Asked,...,1,1,2,1,1,3,3,1,1,3
3,2020-12-10 08:23:48.152,46482.0,NaN,10,No,5,None,35,Not Asked,Not Asked,...,1,1,3,1,2,4,3,1,1,2
4,2020-12-10 08:24:03.600,98800.0,468,5,No,5,None,50,Not Asked,Not Asked,...,1,1,2,1,1,1,4,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10336,2017-01-14 15:50:23.936,36549.0,Not Asked,3,No,1,1,Not Asked,42,NaN,...,6,2,3,3,1,7,6,2,5,13
10337,2017-01-15 01:01:21.212,65000.0,Not Asked,7,Yes,3,5,Not Asked,40,Yes,...,6,2,73,2,1,7,6,3,5,13
10338,2017-01-15 03:25:47.422,85000.0,Not Asked,17,No,17,1,Not Asked,200,No,...,6,2,28,2,1,7,6,3,5,11
10339,2017-01-15 03:37:35.305,90000.0,Not Asked,8,No,3,1,Not Asked,80,Yes,...,6,3,2,4,1,7,6,4,5,13


# DataFrame other_duties

In [67]:
other_duties_columns = pd.read_sql_query("select * from other_duties;", engine).columns
other_duties_columns
other_duties_df = pd.DataFrame(columns=other_duties_columns)

In [68]:
job_df.index = job_df['job_name'].str.len()
job_df = job_df.sort_index(ascending=False).reset_index(drop=True)
job_df

,job_name,job_id
0,DBA (General - splits time evenly between writ...,3
1,"DBA (Development Focus - tunes queries, indexe...",8
2,DBA (Production Focus - build & troubleshoot s...,2
3,"Developer: Business Intelligence (SSRS, PowerB...",1
4,"Developer: App code (C#, JS, etc)",5
5,"DevOps, Sr Software Engineer DBA",15
6,Principal database engineer,14
7,Systems Administrator,19
8,Analytics consultant,23
9,Database Specialist,17


In [69]:
def other_job(x):

    if isinstance(x,float):
        return x

    for key in range(job_df.shape[0]):
        job = job_df.iloc[key]
        x = str(x).replace(job["job_name"],str(job["job_id"]))
    return x

other_duties_df["job_id"] = survey_data['OtherJobDuties'].apply(other_job)

In [70]:
other_duties_df["sgi_id"] = other_duties_df.index + 1

In [71]:
#other_duties_df["job_id"][other_duties_df["job_id"] == "Not Asked"] = None

<ipython-input-71-4404d8a790c2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_duties_df["job_id"][other_duties_df["job_id"] == "Not Asked"] = None


In [77]:
other_duties_df = other_duties_df[other_duties_df["job_id"].isna() != True]
other_duties_df

,job_id,sgi_id
3470,"8, 2, 6",3471
3471,4,3472
3472,2,3473
3473,"7, 8, 3",3474
3474,8,3475
...,...,...
7438,6,7439
7439,7,7440
7440,"7, 1, 6, 4",7441
7441,"8, 3, 6, 4",7442


# DataFrame task

In [42]:
task_columns = pd.read_sql_query("select * from task;", engine).columns
task_columns

Index(['tas_id', 'tas_name'], dtype='object')

In [43]:
task_liste = ["Build scripts & automation tools", "Manual tasks", "Meetings & management", "Not Asked", "On-call 24/7/365", "On-call as part of a rotation", "Projects", "R&D", "Training/teaching"]

task_df = pd.DataFrame({"tas_name" : task_liste})
#task_df.rename_axis(index="tas_id", inplace=True)
task_df["tas_id"] = task_df.index
task_df

,tas_name,tas_id
0,Build scripts & automation tools,0
1,Manual tasks,1
2,Meetings & management,2
3,Not Asked,3
4,On-call 24/7/365,4
5,On-call as part of a rotation,5
6,Projects,6
7,R&D,7
8,Training/teaching,8


# DataFrame task_performed

In [44]:
task_columns = pd.read_sql_query("select * from task;", engine).columns
task_columns    

Index(['tas_id', 'tas_name'], dtype='object')

[]

# DataFrame database_used

In [45]:
database_columns = pd.read_sql_query("select * from database_used;", engine).columns
database_columns

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'survey.database_used' doesn't exist")
[SQL: select * from database_used;]
(Background on this error at: http://sqlalche.me/e/13/f405)

# DataFrame other_database

In [46]:
# Enumeration servant à remplacer les valeurs dans OtherDatabase et PrimaryDatabase
# par des valeurs uniformizé (le nom de l'enum)
# la valeur de l'enum étant un pattern regex pour le remplacement
class EnumDatabases(Enum):
    MS_SQL_SERVER = 'microsoft sql server|ms sql server|ssrs'
    MS_APS = 'parallel data ?warehouse|pdw|aps|azure dw|azure data ?warehouse'
    POSTGRE_SQL = 'postgresql|enterprisedb'
    AZURE_DATA_LAKE = 'data lake'
    AZURE_SQL_DB = 'azure'
    MICROSOFT_ACCESS = 'microsoft access'
    ORACLE = 'oracle'
    MYSQL = 'mysql/mariadb'
    AMAZON_RDS = 'amazon rds'
    MONGO_DB = 'mongodb'
    IBM_DB2 = 'db2|udb|as ?400'
    IBM_NETEZZA = 'netezza'
    CASSANDRA = 'cassandra'
    SQLITE = 'sqlite|db3'
    SAP = 'sap'
    TERADATA = 'teradata'
    ELASTIC_SEARCH = 'elasticsearch'
    COSMO_DB = 'cosmosdb'
    RAVEN_DB = 'raven ?db'
    SNOWFLAKE = 'snowflake'
    HADOOP = 'hadoop|hdfs'
    FIREBIRD = 'firebird'
    FIREBOLT = 'firebolt'
    BIGQUERY = 'big ?query|gbq'
    REDIS = 'redis'
    AMAZON_REDSHIFT = 'redshift'
    AMAZON_AURORA = 'aurora|aws'
    COUCH_DB = 'co[a|u]chdb'
    INFORMIX = 'informix'
    PROGRESS = 'progress|4gl|open ?edge'
    SYBASE = 'sybase'
    INGRES = 'ingres|actian'
    PROVIDEX = 'providex'
    APACHE_MAPR = 'mapr'
    TIBCO_GRAPH_DB = 'tibco'
    VECTOR_WISE = 'informatica'
    ORACLE_BERKELEY = 'berkeley'
    QLIK_SENSE = 'qix'
    ADABAS = 'adabas'
    COUCHBASE = 'couch ?base'
    FOXPRO = 'foxpro'
    EXASOL = 'exasol'
    INTERBASE = 'interbase'
    PERVASIVE = 'pervasive'
    RIAK = 'riak'
    DATACOM = 'datacom'
    GREENPLUM = 'greenplum'
    VERTICA = 'vertica'
    SAP_HANA = 'hana'
    PARADOX = 'paradox'
    GRUPTA_SQLBASE = 'sql ?base'
    DYNAMO_DB = 'dynamo'
    MULTICS_RDS = 'multics'
    CITRIX = 'citrix'
    ALPHAFOUR = 'alpha ?four'
    ROCKET_UNIDATA = 'unidata'
    ROCKET_UNIVERSE = 'universe'
    RAIMA_DBVISTA = 'raima|dbvista'
    AMAZON_DOCUMENT_DB = 'documentdb'
    NEO4J = 'neo4j'
    PICK_OS = 'pick'
    ADS_DB = 'advantage|ads'
    IBM_NOTES_DOMINO = 'domino'
    IBM_IMS = 'ims'
    H2_DB = 'h2'
    FOURD_DB = '4d'
    DATAEASE = 'dataease'
    GRAPH_DB = 'graphdb'
    MONET_DB = 'monetdb'
    MARKLOGIC = 'marklogic'
    KYLIN = 'kylin'
    HPCC = 'hpcc'
    LITE_DB = 'litedb'
    APACHE_IMPALA = 'impala'
    PERVASIVE_BTRIEVE = 'btrieve'
    MEMSQL = 'memsql|singlestore'
    UNISYS = 'unisys'
    INFLUX_DB = 'i[n|m]flux'
    SQL_ANYWHERE = 'sql ?anywhere'
    SPARK = 'spark'
    COCKROACH = 'cockroach'
    GOOGLE_SPANNER = 'spanner'
    GOOGLE_CLOUD_SQL = 'google|gcp'
    SCADA = 'scada'
    GPU_DB = 'kinetica'
    EXTREME_DB = 'extremedb'
    ARANGO_DB = 'arango'
    ROCKS_DB = 'rocks'
    CLIPPER = 'clipper'
    DRUID = 'druid'
    PRESTO_DB = 'presto'
    CLOUDERA_OP_DB = 'cloudera'
    FILEMAKER = 'filemaker'
    APACHE_SOLR = 'solr'
    APACHE_KAFKA = 'kafka'
    APACHE_IGNITE = 'ignite'
    APACHE_HBASE = 'hbase'
    APACHE_PARQUET = 'parquet'
    APACHE_KUDU = 'kudu'
    ALTIBASE = 'altibase'
    SPLUNK = 'splunk'
    SALESFORCE = 'salesforce'
    APACHE_DELTA = 'delta lake'
    APACHE_DERBY = 'derby'
    DATABRICKS = 'databricks'
    CLICKHOUSE = 'clickhouse'
    ORACLE_ESSBASE = 'essbase|hyperion'
    ORIENT_DB = 'orientdb'
    INTERSYSTEMS_CACHE = 'intersystems|cach.'
    IBM_COGNOS = 'tm1'
    IBM_REDBRICK = 'red ?brick'
    IBM_FOCUS = 'focus'
    DB4O = 'db4'
    MUMPS = 'mumps'
    HIVE = 'hive|llap|hdinsight'
    NONSTOP_SQL = 'hp tandem'
    CISCO = 'cisco'
    JADE = 'jade'
    CLUSTRIX = 'clustrix'
    RDB = 'rdb'
    MS_COSMOS_DB = 'cosmo'
    IDMS = 'idms'
    SAS = 'sas'
    DBASE = 'dbase'
    POWER_BI = 'power'
    OTHER = 'other'


def uniformize_database(db: str):
    """Fonction qui retourne pour un nom de base de données, un index (int) correspondant à l'enum

    Args:
        db (str): Nom d'une base de données

    Returns:
        int: Index de l'enum
    """
    for key, database in enumerate(EnumDatabases):
        if re.search('(?:' + database.value + ')',
                     db, flags=re.IGNORECASE) or database.value.find(db) != -1:
            return key + 1
    # to check what was missed
    # print(db)
    return len(EnumDatabases)

NameError: name 'Enum' is not defined

In [ ]:
# manage_staff
for i in range(sondage_item_df.shape[0]):
    if sondage_item_df["manage_staff"].iloc[i] == "Yes":
        sondage_item_df["manage_staff"].iloc[i] = True
    else:
        sondage_item_df["manage_staff"].iloc[i] = False

In [ ]:
# other people
for i in range(sondage_item_df.shape[0]):
    if sondage_item_df["other_people"].iloc[i] == "None":
        sondage_item_df["other_people"].iloc[i] = None
    elif sondage_item_df["other_people"].iloc[i] == "More than 5":
        sondage_item_df["other_people"].iloc[i] = 5
    else:
        None

In [ ]:
#database database_servers
for i in range(sondage_item_df.shape[0]):
    if sondage_item_df["database_servers"].iloc[i] == "Not Asked":
        sondage_item_df["database_servers"].iloc[i] = None
    else:
        None

In [ ]:
#database hours_worked
for i in range(sondage_item_df.shape[0]):
    if sondage_item_df["hours_worked"].iloc[i] == "Not Asked":
        sondage_item_df["hours_worked"].iloc[i] = None
    else:
        None

In [ ]:
#database eduction_computer
for i in range(sondage_item_df.shape[0]):
    if sondage_item_df["education_computer"].iloc[i] == "Not Asked":
        sondage_item_df["education_computer"].iloc[i] = None
    elif sondage_item_df["education_computer"].iloc[i] == "N/A":
        sondage_item_df["education_computer"].iloc[i] = None
    elif sondage_item_df["education_computer"].iloc[i] == "Yes":
        sondage_item_df["education_computer"].iloc[i] = True
    else:
        sondage_item_df["education_computer"].iloc[i] = False

In [ ]:
# Education telecommute
for i in range(sondage_item_df.shape[0]):
    if sondage_item_df["telecommute"].iloc[i] == "Not Asked":
        sondage_item_df["telecommute"].iloc[i] = None
    elif sondage_item_df["telecommute"].iloc[i] == "5 or more":
        sondage_item_df["telecommute"].iloc[i] = 5
    elif sondage_item_df["telecommute"].iloc[i] == "None, or less than 1 day per week":
        sondage_item_df["telecommute"].iloc[i] = 1

In [ ]:
# Education company_employee
#for i in range(sondage_item_df.shape[0]):
#   if sondage_item_df["company_employee"].iloc[i] == "Not Asked":
  #      sondage_item_df["company_employee"].iloc[i] = None

In [178]:
sondage_item_df["database_servers"][sondage_item_df["database_servers"] == "Not Asked"] = None

<ipython-input-178-79ab133f33ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sondage_item_df["database_servers"][sondage_item_df["database_servers"] == "Not Asked"] = None


In [ ]:
#database gender
for i in range(sondage_item_df.shape[0]):
    if sondage_item_df["gender"].iloc[i] == "Not Asked":
        sondage_item_df["gender"].iloc[i] = None
    elif sondage_item_df["gender"].iloc[i] == "Female":
        sondage_item_df["gender"].iloc[i] = "female"
    elif sondage_item_df["gender"].iloc[i] == "Male":
        sondage_item_df["gender"].iloc[i] = "male"
    elif sondage_item_df["gender"].iloc[i] == "Non_binary/third gender":
        sondage_item_df["gender"].iloc[i] = "other"
    elif sondage_item_df["gender"].iloc[i] == "None":
        sondage_item_df["gender"].iloc[i] = None
    else:
        sondage_item_df["gender"].iloc[i] = "na"

# Chargement des données dans la base de données

In [91]:
#employment_status_df.to_sql("employment_status",if_exists="append", con=engine, index=False)


In [197]:
#carrer_df.to_sql("career_plan",if_exists="append", con=engine, index=False)

In [185]:
#hmc_df.to_sql("how_many_companies",if_exists="append", con=engine, index=False)

In [189]:
#certification_df.to_sql("certification",if_exists="append", con=engine, index=False)

In [181]:
#country_df.to_sql("country",if_exists="append", con=engine, index=False)

In [195]:
#looking_job_df.to_sql("looking_job",if_exists="append", con=engine, index=False)

In [193]:
#employment_sector_df.to_sql("employment_sector",if_exists="append", con=engine, index=False)

In [145]:
#sondage_df.to_sql("sondage",if_exists="append", con=engine, index=False)

In [187]:
#education_df.to_sql("education",if_exists="append", con=engine, index=False)

In [191]:
#largest_city_df.to_sql("largest_city",if_exists="append", con=engine, index=False)

In [183]:
#job_df.to_sql("job",if_exists="append", con=engine, index=False)

In [112]:
#task_df.to_sql("task",if_exists="append", con=engine, index=False)

In [114]:
#database_used_df.to_sql("database",if_exists="append", con=engine, index=False)

In [198]:
#sondage_item_df.to_sql("sondage_item",if_exists="append", con=engine, index=False)

In [ ]:
#other_duties_liste_id = sondage_item_df.to_dict(orient="split")["index"]

In [ ]:
.to_sql("sondage_item",if_exists="append", con=engine, index=False)

In [ ]:
.to_sql("sondage_item",if_exists="append", con=engine, index=False)

In [ ]:
a = pd.read_sql_query("select * from sondage;", engine)


In [159]:
liste_table = pd.read_sql_query("show tables;", engine)
liste_table = list(liste_table["Tables_in_survey"])
liste_table

['career_plan',
 'certification',
 'country',
 'database',
 'education',
 'employment_sector',
 'employment_status',
 'how_many_companies',
 'job',
 'largest_city',
 'looking_job',
 'other_database',
 'other_duties',
 'sondage',
 'sondage_item',
 'task',
 'task_performed']